# Packages required

In [1]:
import findspark
findspark.init("/home/juliane_oliveira/spark-2.3.1-bin-hadoop2.7")

In [2]:

import os
from datetime import date
import functools
from IPython.core.display import display, HTML
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import count, isnan, mean, col, countDistinct, format_number, dayofmonth, hour, dayofyear, month, year, weekofyear, date_format, when,datediff, months_between

import re 
from functools import reduce

# Functions

In [3]:
#function to combine all dfs Alternativo
def unionAll(dfs):
    return reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs)

In [4]:
# aumentando a largura do notebook

In [5]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# Starting spark

In [6]:
spark = SparkSession.builder.appName('Sindrome Gripal').getOrCreate()

# Reading and creating the dataframe

In [21]:
path = "/media/juliane_oliveira/My Passport/PAMEpi_datalake/raw_data_covid19_version-21-11-23/data-notificacao_sindrome_gripal/"
path_to_export = "/media/juliane_oliveira/My Passport/GitHub/Data of git computations/"

In [22]:
import glob 
files = glob.glob(path + '*', recursive=False)

In [9]:
# contando arquivos carregados corretamente
print(len(files))

78


In [10]:
# Read data

In [23]:
dfs = []
    
for base in files:
    dfs.append(spark.read.csv(base, header = True, sep=';', encoding="ISO-8859-1", inferSchema = True))
    #print("Number of files read: {}".format(len(dfs)))
    #print('{:*{align}{width}}'.format('Processing files', align='^', width='112'))

In [24]:
dfs = unionAll(dfs)

In [25]:
dfs.limit(5).toPandas()  #dfs.show(5)

,ÿid,dataNotificacao,dataInicioSintomas,dataNascimento,sintomas,profissionalSaude,cbo,condicoes,estadoTeste,dataTeste,...,estadoNotificacao,estadoNotificacaoIBGE,municipioNotificacao,municipioNotificacaoIBGE,excluido,validado,idade,dataEncerramento,evolucaoCaso,classificacaoFinal
0,gINnMy6CdU,2020-06-17T20:08:30.197Z,2020-06-08T03:00:00.000Z,1989-12-26T02:00:00.000Z,"Dor de Garganta, Febre, Outros",Não,null,null,Concluído,2020-06-16T03:00:00.000Z,...,Acre,12,Tarauacá,1200609,undefined,undefined,31,2020-06-17 00:00:00,Em tratamento domiciliar,Confirmado Laboratorial
1,TiABR1GOmb,2020-06-13T03:00:00.000Z,2020-06-07T03:00:00.000Z,1998-11-14T02:00:00.000Z,"Dispneia, Dor de Garganta",Não,null,null,Concluído,2020-06-13T03:00:00.000Z,...,Acre,12,Capixaba,1200179,undefined,undefined,22,None,None,None
2,RG9f49BW1b,2020-06-12T05:00:00.000Z,2020-05-29T05:00:00.000Z,1954-01-28T05:00:00.000Z,"Febre, Outros",Não,null,Doenças cardíacas crônicas,Concluído,2020-06-12T05:00:00.000Z,...,Acre,12,Cruzeiro do Sul,1200203,undefined,undefined,67,None,None,None
3,F66wawHqyP,2020-05-09T05:00:00.000Z,2020-05-02T05:00:00.000Z,1984-07-16T05:00:00.000Z,"Febre, Tosse",Não,null,null,Concluído,2020-05-09T05:00:00.000Z,...,Acre,12,Rio Branco,1200401,undefined,undefined,37,2020-05-27 02:00:00,Cura,Confirmado Laboratorial
4,Gq5GXKSRZY,2020-05-23T03:00:00.000Z,2020-05-10T03:00:00.000Z,1991-09-21T03:00:00.000Z,Outros,Sim,1312 - Gestores e especialistas de operações e...,null,Concluído,2020-05-23T04:00:00.000Z,...,Acre,12,Capixaba,1200179,undefined,undefined,29,None,None,Descartado


# Basic description

In [14]:
print(f'total de linhas: {dfs.count()}')

total de linhas: 65378930


In [15]:
print(f'numero de colunas: {len(dfs.columns)}')

numero de colunas: 29


In [16]:
dfs.printSchema()

root
 |-- ÿid: string (nullable = true)
 |-- dataNotificacao: string (nullable = true)
 |-- dataInicioSintomas: string (nullable = true)
 |-- dataNascimento: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- profissionalSaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- estadoTeste: string (nullable = true)
 |-- dataTeste: string (nullable = true)
 |-- tipoTeste: string (nullable = true)
 |-- resultadoTeste: string (nullable = true)
 |-- paisOrigem: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- estadoIBGE: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- municipioIBGE: string (nullable = true)
 |-- origem: string (nullable = true)
 |-- estadoNotificacao: string (nullable = true)
 |-- estadoNotificacaoIBGE: string (nullable = true)
 |-- municipioNotificacao: string (nullable = true)
 |-- municipioNotificacaoIBGE: string (nullab

## contagem dos nulls / nan

In [12]:
#dfs.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in dfs.columns]).limit(1).toPandas().T

# Harmonizations

## Rename columns

In [26]:
rename_columns = {
    'dataNascimento': 'dt_nasc', 'dataNotificacao': 'dt_ntf', 'dataInicioSintomas': 'dt_sint',
    'dataTeste': 'dt_teste', 'tipoTeste': 'tp_teste', 'resultadoTeste': 'res_teste',
    'classificacaoFinal': 'class_final', 'evolucaoCaso': 'ev_caso', 'estadoNotificacao': 'est_ntf',
}

In [27]:
for old_col, new_col in rename_columns.items():
    dfs = dfs.withColumnRenamed(old_col, new_col)

In [35]:
dfs.printSchema()

root
 |-- ÿid: string (nullable = true)
 |-- dt_ntf: string (nullable = true)
 |-- dt_sint: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- profissionalSaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- estadoTeste: string (nullable = true)
 |-- dt_teste: string (nullable = true)
 |-- tp_teste: integer (nullable = true)
 |-- res_teste: integer (nullable = true)
 |-- paisOrigem: string (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- estado: string (nullable = true)
 |-- estadoIBGE: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- municipioIBGE: string (nullable = true)
 |-- origem: string (nullable = true)
 |-- est_ntf: string (nullable = true)
 |-- estadoNotificacaoIBGE: string (nullable = true)
 |-- municipioNotificacao: string (nullable = true)
 |-- municipioNotificacaoIBGE: string (nullable = true)
 |-- excluido: string (nullabl

## Rename categories of final classification

In [22]:
dfs.groupby('class_final').count().show(truncate=False)

+----------------------------------+--------+
|class_final                       |count   |
+----------------------------------+--------+
|Confirmado por Critério Clínico   |269128  |
|Confirmado                        |2       |
|null                              |34807754|
|Confirmado Clinico-Epidemiologico |11      |
|Confirmado Clínico-Epidemiológico |391007  |
|Confirmado Laboratorial           |13261300|
|Confirmado Clinico-Imagem         |1       |
|Síndrome Gripal Não Especificada  |3424962 |
|Confirmação Clínico Epidemiológico|259     |
|Sindrome Gripal Nao Especificada  |14      |
|Confirmado Clínico-Imagem         |17622   |
|Descartado                        |13190693|
|Confirmação Laboratorial          |16177   |
+----------------------------------+--------+



In [29]:
dfs = dfs.withColumn('class_final', F.when((dfs['class_final'] == 'Confirmado por Critério Clínico'), F.lit(3)) \
                    .when((dfs['class_final'] == 'Confirmado'), F.lit(1)) \
                    .when((dfs['class_final'] == 'Confirmado Clinico-Epidemiologico'), F.lit(2)) \
                    .when((dfs['class_final'] == 'Confirmação Clínico-Epidemiológico'), F.lit(2)) \
                    .when((dfs['class_final'] == 'Confirmado Laboratorial'), F.lit(4)) \
                    .when((dfs['class_final'] == 'Confirmado Clinico-Imagem'), F.lit(5)) \
                    .when((dfs['class_final'] == 'Síndrome Gripal Não Especificada'), F.lit(6)) \
                    .when((dfs['class_final'] == 'Confirmação Clínico Epidemiológico'), F.lit(2)) \
                    .when((dfs['class_final'] == 'Sindrome Gripal Nao Especificada'), F.lit(6)) \
                    .when((dfs['class_final'] == 'Confirmado Clínico-Imagem'), F.lit(5)) \
                    .when((dfs['class_final'] == 'Descartado'), F.lit(7)) \
                    .when((dfs['class_final'] == 'Confirmação Laboratorial'), F.lit(4)))

'Confirmado' -- > 1

'Confirmado Clinico-Epidemiologico' --> 2

'Confirmado por Critério Clínico' --> 3

'Confirmado Laboratorial' --> 4

'Confirmado Clinico-Imagem' --> 5

'Síndrome Gripal Não Especificada' --> 6

'Descartado' --> 7

In [24]:
dfs.groupby('class_final').count().show(truncate=False)

+-----------+--------+
|class_final|count   |
+-----------+--------+
|null       |35198761|
|1          |2       |
|6          |3424976 |
|3          |269128  |
|5          |17623   |
|4          |13277477|
|7          |13190693|
|2          |270     |
+-----------+--------+



## Rename categories of laboratory test result

In [30]:
dfs = dfs.withColumn('res_teste', F.when((dfs['res_teste'] == 'Positivo'), F.lit(1)) \
                   .when((dfs['res_teste'] == 'undefined') | (dfs['res_teste'] == 'Inconclusivo ou Indefinido'), F.lit(9)) \
                   .when((dfs['res_teste'] == 'Negativo'), F.lit(2)))

## Rename cathegories type of test

In [26]:
dfs.groupby('tp_teste').count().show(truncate=False)

+-----------------------------------------------------+--------+
|tp_teste                                             |count   |
+-----------------------------------------------------+--------+
|RT-PCR                                               |25768453|
|Imunoensaio por Eletroquimioluminescência - ECLIA IgG|63260   |
|Imunoensaio por Eletroquimioluminescência  ECLIA    |17025   |
|null                                                 |540     |
|Quimioluminescência - CLIA                           |26989   |
|Teste rápido                                         |80      |
|Enzimaimunoensaio - ELISA IgM                        |48701   |
|Enzimaimunoensaio  ELISA                            |11788   |
|TESTE RÁPIDO - ANTICORPO                             |18943283|
|Concluído                                            |1       |
|TESTE RÁPIDO - ANTÍGENO                              |10933254|
|undefined                                            |6       |
|null                    

In [32]:
dfs = dfs.withColumn('tp_teste', F.when((dfs['tp_teste'] == 'RT-PCR'), F.lit(1)) \
                    .when((dfs['tp_teste'] == 'Imunoensaio por Eletroquimioluminescência - ECLIA IgG') | (dfs['tp_teste'] == 'Imunoensaio por Eletroquimioluminescência  ECLIA'), F.lit(2)) \
                    .when((dfs['tp_teste'] == 'Quimioluminescência - CLIA'), F.lit(3)) \
                    .when((dfs['tp_teste'] == 'Teste rápido'), F.lit(4)) \
                    .when((dfs['tp_teste'] == 'Enzimaimunoensaio - ELISA IgM'), F.lit(5)) \
                    .when((dfs['tp_teste'] == 'Enzimaimunoensaio   ELISA'), F.lit(5)) \
                    .when((dfs['tp_teste'] == 'TESTE RÁPIDO - ANTICORPO'), F.lit(6)) \
                    .when((dfs['tp_teste'] == 'Concluído'), F.lit(7)) \
                    .when((dfs['tp_teste'] == 'TESTE RÁPIDO - ANTÍGENO'), F.lit(4)) \
                    .when((dfs['tp_teste'] == 'undefined'), F.lit(9)))

## Rename cathegories of the evolution of the case

In [23]:
#dfs.groupby('ev_caso').count().show(truncate=False)

In [33]:
dfs = dfs.withColumn('ev_caso', F.when((dfs['ev_caso'] == 'Óbito'), F.lit(1)) \
                  .when((dfs['ev_caso'] == 'Cura'), F.lit(2)) \
                  .when((dfs['ev_caso'] == 'Internado em UTI'), F.lit(3)) \
                  .when((dfs['ev_caso'] == 'Internado'), F.lit(4)) \
                  .when((dfs['ev_caso'] == 'Ignorado'), F.lit(9)) \
                  .when((dfs['ev_caso'] == 'Cancelado'), F.lit(5)) \
                  .when((dfs['ev_caso'] == 'Em tratamento domiciliar'), F.lit(6)))

In [ ]:
dfs.groupby('ev_caso').count().show()

## Rename cathegory of gender

In [ ]:
dfs.groupby('sexo').count().show()

In [34]:
dfs = dfs.withColumn('sexo', F.when((dfs['sexo'] == 'Feminino'), F.lit(1)) \
                   .when((dfs['sexo'] == 'Masculino') | (dfs['sexo'] == 'M'), F.lit(2)) \
                   .when((dfs['sexo'] == 'Indefinido') | (dfs['sexo'] == 'undefined'), F.lit(9)))

In [ ]:
dfs.groupby('sexo').count().show()

# symptoms 

In [59]:
dfs.groupby('sintomas').count().show()

+--------------------+------+
|            sintomas| count|
+--------------------+------+
|Outros, Tosse, Di...|  1129|
|Tosse, Outros, Do...| 30177|
|Febre, Dor de Gar...| 36792|
|Dor de Cabeça, Ou...|600446|
|Febre, Tosse, Dor...|  2953|
|Febre, Dor de Cab...|  1516|
|Coriza, Dor de Ca...|   398|
|Febre, Tosse, Dor...|  6297|
|Dor de Cabeça, Fe...|  2641|
|Dor de Cabeça, Di...|   240|
|Dor de Cabeça, Co...|   771|
|Outros, Dispneia,...|   678|
|Dor de Garganta, ...|    76|
|Tosse, Distúrbios...| 16787|
|Dor de Garganta, ...|  2279|
|Tosse, Dor de Cab...|   220|
|Dor de Cabeça, Di...|  2304|
|Tosse, Febre, Dor...|  8546|
|Dispneia, Dor de ...|     1|
|Febre, Dor de Gar...|    62|
+--------------------+------+
only showing top 20 rows



# Periodo de cobertura da base

In [13]:
list_var_date = ['dt_sint', 'dt_nasc', 'dt_ntf', 'dt_teste']

In [ ]:
for col in list_var_date:
    dfs = dfs.withColumn(col, F.col(col).cast(T.DateType()))

## Date of notification

In [25]:
#dfs.groupBy(year(dfs['dataNotificacao']),month(dfs['dataNotificacao'])).count().show(100)

In [15]:
dfs.groupBy(year(dfs['dataNotificacao'])).count().show(100)

+---------------------+--------+
|year(dataNotificacao)|   count|
+---------------------+--------+
|                 1959|       1|
|                 2003|       1|
|                 1974|       1|
|                 2015|       1|
|                 2022|       1|
|                 1942|       1|
|                 null|   35991|
|                 1988|       2|
|                 1994|       1|
|                 1973|       1|
|                 2019|       1|
|                 1971|       1|
|                 1991|       1|
|                 1989|       1|
|                 1998|       2|
|                 1965|       1|
|                 2020|32152435|
|                 1985|       2|
|                 1970|     304|
|                 2009|       1|
|                 1995|       1|
|                 1980|       1|
|                 1983|       2|
|                 2000|       1|
|                 2011|       1|
|                 1976|       2|
|                 2008|       1|
|         

In [50]:
print('percentage of nulls on date of notification' ,35991*100/65378930)

percentage of nulls on date of notification 0.05504984556951299


In [32]:
df1 = dfs.filter((dfs['dataNotificacao'] >= '2020-02-20') & (dfs['dataNotificacao'] < date.today()))

In [33]:
print('number of valid values', df1.count())

65339643

In [49]:
print( 'percentage of inconsistency', 3296*100/65378930)

percentage of inconsistency 0.005041379539249113


## Date of symptoms

In [16]:
dfs.groupBy(year(dfs['dataInicioSintomas'])).count().show(1000)

+------------------------+--------+
|year(dataInicioSintomas)|   count|
+------------------------+--------+
|                    1959|       2|
|                    1990|       8|
|                    1975|      60|
|                    2025|       3|
|                    1977|     188|
|                    2027|       2|
|                    2200|       1|
|                    2003|      17|
|                    2007|       8|
|                    2052|       1|
|                    2018|     339|
|                    1974|       5|
|                    2015|     109|
|                    2023|       3|
|                    2006|      10|
|                    2920|       1|
|                    2022|      29|
|                    1978|       2|
|                    1961|       1|
|                    2013|      47|
|                    1899|     135|
|                    3020|       2|
|                    null| 1092901|
|                    2102|       2|
|                    1952|  

In [44]:
dfs.filter((dfs['dataInicioSintomas'] >= '2020-02-20') & (dfs['dataInicioSintomas'] < date.today())).count()

64209071

In [45]:
print('percentage of valid values in date of first symptoms', 64209071*100/65378930)

percentage of valid values in date of first symptoms 98.21064829295922


In [23]:
print('percentage of null in date of first symptoms' , 1092901*100/65378930)

1.6716410011604657

In [52]:
76958*100/(65378930)

0.11771070588031955

##  Date of test

In [17]:
dfs.groupBy(year(dfs['dataTeste'])).count().show(100) 

+---------------+--------+
|year(dataTeste)|   count|
+---------------+--------+
|           1959|       4|
|           1990|      24|
|           1975|      67|
|           1977|     201|
|           2003|       7|
|           2007|      11|
|           2018|      16|
|           1974|      11|
|           2015|       8|
|           2023|       1|
|           1955|       6|
|           2006|       8|
|           1978|      14|
|           2022|       2|
|           1961|       2|
|           2013|      10|
|           1939|       5|
|           1899|     181|
|           null|12082513|
|           1952|       2|
|           1956|       2|
|           1988|      15|
|           1997|       7|
|           1994|      80|
|           1968|      14|
|           2014|       9|
|           1951|       2|
|           1973|      12|
|           1979|      15|
|           1946|       3|
|           1947|       1|
|           2019|     221|
|           1971|      10|
|           1950|       3|
|

In [53]:
dfs.filter((dfs['dataTeste'] >= '2020-02-20') & (dfs['dataTeste'] < date.today())).count()

53259570

In [54]:
53259570*100/65378930

81.46289637961343

In [57]:
36847*100/65378930

0.05635913588674516

In [58]:
12082513*100/65378930

18.480744484499823

# Analysis for the paper 2

In [37]:
dfs.printSchema()

root
 |-- ÿid: string (nullable = true)
 |-- dt_ntf: string (nullable = true)
 |-- dt_sint: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- profissionalSaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- estadoTeste: string (nullable = true)
 |-- dt_teste: string (nullable = true)
 |-- tp_teste: integer (nullable = true)
 |-- res_teste: integer (nullable = true)
 |-- paisOrigem: string (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- estado: string (nullable = true)
 |-- estadoIBGE: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- municipioIBGE: string (nullable = true)
 |-- origem: string (nullable = true)
 |-- est_ntf: string (nullable = true)
 |-- estadoNotificacaoIBGE: string (nullable = true)
 |-- municipioNotificacao: string (nullable = true)
 |-- municipioNotificacaoIBGE: string (nullable = true)
 |-- excluido: string (nullabl

## Filtering valid date range

In [38]:
df1 = dfs.filter((dfs['dt_ntf'] >= '2020-02-20') & (dfs['dt_ntf'] < date.today()))

In [51]:
df1.count()

65339643

In [48]:
df1.groupby('class_final').count().show(truncate=False)

+-----------+--------+
|class_final|count   |
+-----------+--------+
|null       |35175255|
|1          |2       |
|6          |3422378 |
|3          |268952  |
|5          |17612   |
|4          |13271106|
|7          |13184068|
|2          |270     |
+-----------+--------+



In [52]:
13184068 + 3422378 # descartado e inderteminado

16606446

In [53]:
2+268952 +17612+13271106+270 # confirmado

13557942

### Fiter confirmed

'Confirmado' -- > 1

'Confirmado Clinico-Epidemiologico' --> 2

'Confirmado por Critério Clínico' --> 3

'Confirmado Laboratorial' --> 4

'Confirmado Clinico-Imagem' --> 5

'Síndrome Gripal Não Especificada' --> 6

'Descartado' --> 7

In [46]:
df1_conf = df1.filter((df1['class_final'] == 1) | (df1['class_final'] == 2) | (df1['class_final'] == 3)| (df1['class_final'] == 4)| (df1['class_final'] == 5))
        

In [47]:
df1_conf.count()

13557942

In [49]:
df1_conf.groupby('class_final').count().show(truncate=False)

+-----------+--------+
|class_final|count   |
+-----------+--------+
|1          |2       |
|3          |268952  |
|5          |17612   |
|4          |13271106|
|2          |270     |
+-----------+--------+



In [ ]:
'Óbito' --> 1
'Cura' --> 2
'Internado em UTI' --> 3
'Internado'), F.lit(3)) \
                  .when((dfs['ev_caso'] == 'Ignorado'), F.lit(9)) \
                  .when((dfs['ev_caso'] == 'Cancelado'), F.lit(4)) \
                  .when((dfs['ev_caso'] == 'Em tratamento domiciliar'), F.lit(5)))

In [50]:
df1_conf.groupby('ev_caso').count().show()

+-------+-------+
|ev_caso|  count|
+-------+-------+
|   null|2074950|
|      1| 105319|
|      3|  28503|
|      5|1179083|
|      9| 655867|
|      4| 338059|
|      2|9176161|
+-------+-------+

